In [ ]:
"""
Mon application modifiée pour l'anniversaire
"""

import pygame
import sys
import os
import random
import datetime
import moviepy
from moviepy import VideoFileClip  # Importer moviepy pour lire la vidéo

# Hardcoded metadata values
app_name = "Joyeux Anniversaire!"

# Constantes
WHITE = (255, 255, 255)
SCREEN_WIDTH, SCREEN_HEIGHT = 400, 600
BIRD_X = 50
BIRD_Y = 300
GRAVITY = 0.25
FLAP_STRENGTH = -5
PIPE_SPEED_INIT = -4
PIPE_GAP = 150
PIPE_GAP_MIN = 100  # Valeur minimale de l'écart
PIPE_GAP_MAX = 180  # Valeur maximale de l'écart
SPEED_INCREASE_INTERVAL = 3000
SCORE_INTERVAL_FOR_SPEED_INCREASE = 5

FONT_NAME = pygame.font.match_font('arial')

# **Supprimer la variable 'today' puisque nous n'en avons plus besoin**
# today = datetime.datetime.now().day

# Fonctions utilitaires
def draw_text(surf, text, size, x, y, color=(255, 255, 255), font_name=FONT_NAME):
    font = pygame.font.Font(font_name, size)
    text_surface = font.render(text, True, color)
    text_rect = text_surface.get_rect()
    text_rect.midtop = (x, y)
    surf.blit(text_surface, text_rect)

def create_snowflakes(num_flakes):
    return [[random.randint(0, SCREEN_WIDTH),
             random.randint(0, SCREEN_HEIGHT),
             random.randint(2, 5)] for _ in range(num_flakes)]

def update_and_draw_snowflakes(screen, snowflakes):
    for flake in snowflakes:
        flake[1] += flake[2]
        pygame.draw.circle(screen, WHITE, (flake[0], flake[1]), flake[2])
        if flake[1] > SCREEN_HEIGHT:
            flake[0] = random.randint(0, SCREEN_WIDTH)
            flake[1] = -5
            flake[2] = random.randint(2, 5)

def draw_button(surf, text, x, y, width, height):
    mouse = pygame.mouse.get_pos()
    click = pygame.mouse.get_pressed()
    button_clicked = False

    if x + width > mouse[0] > x and y + height > mouse[1] > y:
        pygame.draw.rect(surf, (200, 200, 200), (x, y, width, height))
        if click[0] == 1:
            button_clicked = True
    else:
        pygame.draw.rect(surf, (170, 170, 170), (x, y, width, height))

    draw_text(surf, text, 20, x + width / 2, y + 10)
    return button_clicked

def start_screen(screen):
    stars = create_stars(25)  # Création des étoiles
    snowflakes = create_snowflakes(100)
    intro_text = [
        "Bienvenue au jeu d'anniversaire!",
        "Aidez le personnage à atteindre",
        "le score de 10 pour une surprise spéciale!",
        "",
        "Utilisez la flèche du haut pour monter,",
        "et celle du bas pour descendre.",
        "Bonne chance!"
    ]

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        screen.fill((15, 5, 107))
        draw_stars(screen, stars)  # Dessiner les étoiles
        update_and_draw_snowflakes(screen, snowflakes)

        draw_text(screen, "Joyeux Anniversaire!", 40, SCREEN_WIDTH / 2,
                  SCREEN_HEIGHT / 10, color=(255, 50, 0)) 
        y_offset = 150
        for line in intro_text:
            draw_text(screen, line, 20, SCREEN_WIDTH / 2, y_offset)
            y_offset += 30

        if draw_button(screen, "Commencer", 150, 450, 100, 50):
            break

        pygame.display.flip()
        pygame.time.Clock().tick(15)

def game_over_screen(screen, final_score):
    stars = create_stars(25)  # Création des étoiles
    snowflakes = create_snowflakes(100)

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        screen.fill((15, 5, 107))
        draw_stars(screen, stars)  # Dessiner les étoiles
        update_and_draw_snowflakes(screen, snowflakes)  # Dessiner les flocons de neige

        # Redessiner le texte "Game Over" et le score final dans la boucle
        draw_text(screen, "Game Over", 48, SCREEN_WIDTH / 2, SCREEN_HEIGHT / 4)
        score_text = f"Score Final: {final_score}"
        draw_text(screen, score_text, 36, SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 + 50)

        if draw_button(screen, "Rejouer", 100, 350, 100, 50):
            return True  # Indique de recommencer le jeu
        if draw_button(screen, "Quitter", 200, 350, 100, 50):
            return False  # Indique de quitter le jeu

        pygame.display.flip()
        pygame.time.Clock().tick(15)

def create_stars(num_stars):
    return [[random.randint(0, SCREEN_WIDTH),
             random.randint(0, SCREEN_HEIGHT)] for _ in range(num_stars)]

def draw_stars(screen, stars):
    for star in stars:
        pygame.draw.circle(screen, (255, 255, 0), star, random.randint(1, 3))

def wait_for_key():
    waiting = True
    while waiting:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                waiting = False

# **Ajouter une fonction pour jouer la vidéo**
def play_video(screen, video_path):
    clip = VideoFileClip(video_path)
    # clip = moviepy.video.fx.all.resize(clip, height=SCREEN_HEIGHT, width=SCREEN_WIDTH)  # Ajustez la hauteur de la vidéo à l'écran
    clip = clip.resized(height=SCREEN_HEIGHT, width=SCREEN_WIDTH)
    clock = pygame.time.Clock()

    for frame in clip.iter_frames(fps=30, dtype="uint8"):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
        frame_surface = pygame.surfarray.make_surface(frame.swapaxes(0, 1))
        screen.blit(frame_surface, (0, 0))
        pygame.display.update()
        clock.tick(30)
    clip.close()

def main():
    pygame.init()
    pygame.display.set_caption(app_name)
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

    start_screen(screen)

    # Initialisation de la logique du jeu
    bird_img = pygame.image.load('/Users/souchaud/Documents/Autre/2023_12_15_Mission_Noel/resources/pere_noel.png').convert_alpha()
    bird_img = pygame.transform.scale(bird_img, (30, 30))
    pipe_img = pygame.Surface((50, 300))
    pipe_img.fill((255, 0, 0))

    stars = create_stars(10)  # Choisissez le nombre d'étoiles

    # **Supprimer la variable 'level_message_displayed'**
    # level_message_displayed = False

    # **Ajouter une variable pour vérifier si la vidéo a été jouée**
    video_played = False

    # Déclaration de la variable pour suivre le dernier niveau de score où la vitesse a été augmentée
    last_speed_increase_score = 0

    running = True
    while running:
        bird_y = BIRD_Y
        bird_velocity = 0
        pipes = [[300, random.randint(100, 400), False, random.randint(PIPE_GAP_MIN, PIPE_GAP_MAX)]]
        score = 0
        pipe_speed = PIPE_SPEED_INIT
        snowflakes = create_snowflakes(25)

        while running:
            # Logique du jeu
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_UP:
                        bird_velocity = FLAP_STRENGTH
                    elif event.key == pygame.K_DOWN:
                        bird_velocity = -FLAP_STRENGTH

            bird_velocity += GRAVITY
            bird_y += bird_velocity

            screen.fill((15, 5, 107))
            draw_stars(screen, stars)  # Dessiner les étoiles
            update_and_draw_snowflakes(screen, snowflakes)
            screen.blit(bird_img, (BIRD_X, bird_y))

            # Gestion des tuyaux
            for pipe in pipes:
                pipe[0] += pipe_speed
                pipe_x, pipe_y, pipe_passed, pipe_gap = pipe

                screen.blit(pipe_img, (pipe_x, pipe_y))  # Partie inférieure
                screen.blit(pygame.transform.flip(pipe_img, False, True),
                            (pipe_x, pipe_y - pipe_gap - 300))  # Partie supérieure

                # Générer de nouveaux tuyaux
                if pipe[0] < -50:
                    new_gap = random.randint(PIPE_GAP_MIN, PIPE_GAP_MAX)  # Nouvel écart aléatoire
                    new_pipe_y = random.randint(100, 400)
                    pipes.append([400, new_pipe_y, False, new_gap])
                    pipes.pop(0)

                # Vérifier si le tuyau a été passé pour augmenter le score
                if pipe[0] + pipe_img.get_width() < BIRD_X and not pipe_passed:
                    score += 1
                    pipe[2] = True  # Marquer le tuyau comme passé

            draw_text(screen, f"Score: {score}", 24, SCREEN_WIDTH - 100, 50)

            # Vérifier si le score a atteint un nouveau multiple de SCORE_INTERVAL_FOR_SPEED_INCREASE
            if score // SCORE_INTERVAL_FOR_SPEED_INCREASE > last_speed_increase_score:
                pipe_speed -= 1  # Augmenter la vitesse des tuyaux
                last_speed_increase_score = score // SCORE_INTERVAL_FOR_SPEED_INCREASE

            # **Ajouter la condition pour jouer la vidéo lorsque le score atteint 10**
            if score >= 1 and not video_played:
                play_video(screen, '/Users/souchaud/Documents/Autre/2023_12_15_Mission_Noel/resources/test_video.avi')  # Spécifiez le chemin vers votre vidéo
                video_played = True  # Éviter de rejouer la vidéo
                # Vous pouvez ajouter une pause ou un message après la vidéo si vous le souhaitez

            bird_rect = bird_img.get_rect(topleft=(BIRD_X, bird_y))
            for pipe in pipes:
                pipe_rect = pipe_img.get_rect(topleft=(pipe[0], pipe[1]))
                pipe_rect_top = pipe_img.get_rect(topleft=(pipe[0], pipe[1] - pipe[3] - 300))
                if (bird_rect.colliderect(pipe_rect) or bird_rect.colliderect(pipe_rect_top) or
                        bird_y >= SCREEN_HEIGHT - 30 or bird_y <= 0):
                    running = False
                    break

            pygame.display.update()
            pygame.time.Clock().tick(30)

        if not game_over_screen(screen, score):
            break  # Quitter si l'utilisateur choisit de ne pas recommencer

        running = True  # Recommencer le jeu
        video_played = False  # Réinitialiser pour le prochain jeu

    pygame.quit()

if __name__ == "__main__":
    main()